In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

all_systems_csv_path = "/Users/iyngkarrankumar/Documents/Misc/Tracking models/data/all_systems.csv"
notable_models_csv_path = "data/notable_ai_models.csv"
large_scale_models_csv_path = "data/large_scale_ai_models.csv"

In [ ]:
def half_year_bin(date):
    #CHATGPT generated

    if date.month <= 6:
        return f'{date.year}-H1'
    else: 
        return f'{date.year}-H2'

def year_bin(date):
    return date.year


def exponential_model(x,a,b):
    return a*np.exp(b*(x-2017))

def geometric_model(x,a,r):
    return a*r**(x-2017)

## nan ratios

In [ ]:

all_systems_csv_path = "/Users/iyngkarrankumar/Documents/Misc/Tracking models/data/all_systems.csv"
notable_models_csv_path = "data/notable_ai_models.csv"
large_scale_models_csv_path = "data/large_scale_ai_models.csv"

ALL_SYSTEMS_DATA = pd.read_csv(all_systems_csv_path)
NOTABLE_SYSTEMS_DATA = pd.read_csv(notable_models_csv_path)
LARGE_SCALE_DATA = pd.read_csv(large_scale_models_csv_path)

AS_nan_ratio = (ALL_SYSTEMS_DATA['Training compute (FLOP)'].isna().sum())/len(ALL_SYSTEMS_DATA)
NS_nan_ratio = (NOTABLE_SYSTEMS_DATA['Training compute (FLOP)'].isna().sum())/len(NOTABLE_SYSTEMS_DATA)
LS_nan_ratio = (LARGE_SCALE_DATA['Training compute (FLOP)'].isna().sum())/len(LARGE_SCALE_DATA)


print(AS_nan_ratio,NS_nan_ratio,LS_nan_ratio)

## all systems v.s. large scale

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


random_seed = 42
np.random.seed(random_seed)


all_systems_csv_path = "/Users/iyngkarrankumar/Documents/Misc/Tracking models/data/all_systems.csv"
notable_models_csv_path = "data/notable_ai_models.csv"
large_scale_models_csv_path = "data/large_scale_ai_models.csv"

In [ ]:
def half_year_bin(date):
    #CHATGPT generated

    if date.month <= 6:
        return f'{date.year}-H1'
    else: 
        return f'{date.year}-H2'

def year_bin(date):
    return date.year

In [ ]:
ALL_SYSTEMS_DATA = pd.read_csv(all_systems_csv_path)
NOTABLE_SYSTEMS_DATA = pd.read_csv(notable_models_csv_path)
LARGE_SCALE_DATA = pd.read_csv(large_scale_models_csv_path)

start_year = 2017

DATASHEET_LIST = [ALL_SYSTEMS_DATA,LARGE_SCALE_DATA]

for DATASHEET in DATASHEET_LIST:
    DATASHEET['Publication date'] = pd.to_datetime(DATASHEET['Publication date'])
    DATASHEET = DATASHEET[DATASHEET['Publication date'] > f'{start_year}-01-01'] #data filtering


    DATASHEET['Publication bin'] = DATASHEET['Publication date'].apply(year_bin)
    DATASHEET.dropna(subset=['Training compute (FLOP)'],inplace=True)
    DATASHEET['log Training compute'] = np.log10(DATASHEET['Training compute (FLOP)'])


fig,ax= plt.subplots()

ax.scatter(ALL_SYSTEMS_DATA['Publication bin'],ALL_SYSTEMS_DATA['log Training compute'])

## Exploring large-scale model distributions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

all_systems_csv_path = "/Users/iyngkarrankumar/Documents/Misc/Tracking models/data/all_systems.csv"
notable_models_csv_path = "data/notable_ai_models.csv"
large_scale_models_csv_path = "data/large_scale_ai_models.csv"


In [ ]:
LARGE_SCALE_DATA = pd.read_csv(large_scale_models_csv_path)

LARGE_SCALE_DATA['Publication date'] = pd.to_datetime(LARGE_SCALE_DATA['Publication date'])
start_year = 2017
LARGE_SCALE_DATA = LARGE_SCALE_DATA[LARGE_SCALE_DATA['Publication date'] > f'{start_year}-01-01']

#binning
LARGE_SCALE_DATA['Publication bin'] = LARGE_SCALE_DATA['Publication date'].apply(year_bin)

COMPUTE_DATA = LARGE_SCALE_DATA.dropna(subset=['Training compute (FLOP)'])
COMPUTE_DATA['log Training compute'] = np.log10(COMPUTE_DATA['Training compute (FLOP)'])

In [ ]:
DOUBLE_2024 = True

years = list(reversed(COMPUTE_DATA['Publication bin'].unique()))
years.insert(1,2018)
years.insert(2,2019)

fig,axs = plt.subplots(nrows=len(years),ncols=1,figsize=(7,11),sharex=True)
bin_range = (22,27)
bins=np.arange(bin_range[0],bin_range[-1],1)

HISTOGRAM_DATA_DF = pd.DataFrame(columns=years)
for bin in bins[:-1]:
    HISTOGRAM_DATA_DF.loc[f'{bin}-{bin+1}'] = [np.nan]*len(years)

for idx,year in enumerate(years):
    ax = axs[idx]
    filtered_df = COMPUTE_DATA[COMPUTE_DATA['Publication bin']==year] #year df
    
    if year==2024 and DOUBLE_2024: filtered_df = pd.concat([filtered_df,filtered_df],ignore_index=True)

    #histogram data
    bin_count = np.histogram(filtered_df['log Training compute'],bins=bins,range=bin_range)[0]
    for idx,bin in enumerate(bins[:-1]): #-1 index to sort out indexing problems
        HISTOGRAM_DATA_DF.loc[f'{bin}-{bin+1}',year] = int(bin_count[idx])

    #plot
    filtered_df['log Training compute'].plot(kind='hist',bins=bins,range=bin_range,edgecolor='black',ax=ax)
    ax.set_xlabel('');ax.set_ylabel('')
    ax.set_xlim(bin_range)
    ax.tick_params(axis='y',labelsize=12)
    #if idx==0:     ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))

    ax.set_title(f'Year {year}, n={len(filtered_df)}',fontsize=15)
    ax.grid(alpha=0.5)
    ax.set_ylim([0,40])



fig.text(0.5, -0.04, 'Log Compute ($10^X$)', ha='center', fontsize=15)
fig.text(-0.04, 0.5, 'Frequency', va='center', rotation='vertical', fontsize=15)
plt.xticks(bins,fontsize=15)
plt.subplots_adjust(hspace=10)
plt.tight_layout(rect=[0.04, 0.04, 1, 1])



#### Fitting distributions

In [ ]:
PLOT_PROBS = False


years_to_fit = [2022,2023,2024] #just from eyeballing plots for now
NORMED_FIT_DATA = [] #to store normed fit data for later plotting
NORM_FACTORS = []
ALPHA = [] #to store shape parameter
BETA = [] #to store rate parameter
const = 1e-4

for year in years_to_fit:

    #adding values
    fit_data = HISTOGRAM_DATA_DF[year]
    fit_data = fit_data.drop('22-23')
    new_data = pd.Series([0,0,0],index=['26-27','27-28','28-29'])
    fit_data = pd.concat([fit_data,new_data])

    #data structure processing
    fit_data_arr = np.array(fit_data) + const
    norm_factor = fit_data_arr.sum(); NORM_FACTORS.append(norm_factor)
    normed_fit_data = (fit_data_arr/norm_factor)+const #const to prevent fit errors
    NORMED_FIT_DATA.append(normed_fit_data)

    shape,_,scale = stats.gamma.fit(fit_data_arr,floc=0)

    ALPHA.append(shape)
    BETA.append(scale)

x_labels = fit_data.index

fig,axs=plt.subplots(ncols=3,figsize=(12,6),sharey=True)
for idx,ax in enumerate(axs):
    #ax.hist(normed_fit_data)

    x=np.linspace(0.2,1.2,len(fit_data))
    ax.set_xticks(x)

    if PLOT_PROBS:
        ax.bar(x,NORMED_FIT_DATA[idx],width=0.1,alpha=0.5,color='g',label='Data') #plot data

        fitted_pdf = stats.gamma.pdf(x,ALPHA[idx],0,BETA[idx])
        ax.plot(x,fitted_pdf,'r-',lw=2,label='Fitted gamma distribution')

        ax.set_xticklabels(x_labels,rotation=45)
        if idx==1: ax.legend()
        if idx==0: ax.set_ylabel('Probability')

    else: 
        ax.bar(x,NORM_FACTORS[idx]*NORMED_FIT_DATA[idx],width=0.1,alpha=0.5,color='g',label='Data') #plot data

        fitted_pdf = stats.gamma.pdf(x,ALPHA[idx],0,BETA[idx])
        ax.plot(x,NORM_FACTORS[idx]*fitted_pdf,'r-',lw=2,label='Fitted gamma distribution')

        ax.set_xticklabels(x_labels,rotation=45)
        if idx==1: ax.legend()
        if idx==0: ax.set_ylabel('Frequency')

    ax.set_title(f'Year: {years_to_fit[idx]}')

fig.suptitle('Gamma distribution frequency predictions')
